In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%load_ext tensorboard

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [4]:
from PIL import Image
from IPython.display import display

import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from get_frames import get_frames
from build_model_ed import build_model_1, build_model_2
from build_model_lstm import build_model_lstm_1, build_model_lstm_2, build_model_lstm_3
from create_encoder_decoder_data import process_image, process_dataset, get_full_path
from get_encoder_decoder import get_encoder_decoder
from sliding_window import sliding_window

In [5]:
#gpu check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
stride_step_list = [50,100,160,200]
unit_numb_list = [5,10,20,30]
stateful_list = [False]
sliding_window_settings_dict = {
    stride_step_list[0]:[(6,1,0),(6,6,3)],
    stride_step_list[1]:[(6,1,0),(6,6,3),(4,1,0),(4,4,2)],
    stride_step_list[2]:[(6,1,0),(6,6,3),(4,1,0),(4,4,2)],
    stride_step_list[3]:[(6,1,0),(6,6,3),(4,1,0),(4,4,2),(2,1,0),(2,2,1)],    
}

In [7]:
test_size = 0.1
norm_factor=1
dtype = 'uint8'

ifDense = True
ifSummary = False
epochs = 2000
validation_split = 0.1
patience_lr = 2
patience_early_stop = 4

loss = 'mean_squared_error'
learning_rate = 0.1
optimizer_name = "Adadelta"

In [ ]:
for stateful in stateful_list:
    for stride_step in stride_step_list:
        sliding_window_settings_list = sliding_window_settings_dict[stride_step]
        for unit_numb in unit_numb_list:
            #load encoded data
            file_name = f'save_data/encoded_model_2/encoded_{unit_numb}_{stride_step}_ts_{test_size}_{dtype}_norm_{norm_factor}.pkl'
            with open(file_name, 'rb') as f:
                frames_data_encoded = pickle.load(f)
            #shuffle
            np.random.shuffle(frames_data_encoded)
            
            for sliding_window_settings in sliding_window_settings_list:
                window_size, window_size_predicted, overlap = sliding_window_settings
                
                #get sliding window data for lstm input/output
                x_train_lstm, y_train_lstm = sliding_window(frames_data_encoded, window_size, window_size_predicted, overlap)
                
                if window_size_predicted==1:
                    many_to_many = False
                    cells_list = [unit_numb*2]
                else:
                    many_to_many = True
                    cells_list = [unit_numb*2, unit_numb*2]

                if stateful:
                    #one batch encapsulates a whole image
                    batch_size = x_train_lstm.shape[0]//frames_data_encoded.shape[0]
                else:
                    batch_size = 10

                input_shape = (None, unit_numb)

                folder = f'stride_{stride_step}/lstm/lstm_state_{stateful}'                
                cells_list_str = '_'.join(str(x) for x in cells_list)
                model_name = f'model_lstm_win_{window_size}_{window_size_predicted}_ovrp_{overlap}_un_{unit_numb}_c_{cells_list_str}'               
                print(model_name)
                
                lstm_settings_dict = {
                    'unit_numb':unit_numb,
                    'window_size_predicted':window_size_predicted,
                    'window_size':window_size,
                    'overlap':overlap,
                    'cells_list':cells_list,
                    'many_to_many':many_to_many,
                    'ifDense':ifDense,
                    'batch_size':batch_size,
                    'stateful':stateful,
                    'stride_step':stride_step
                }
                
                #save lstm settings to json
                file_settings_name = f'{model_name}_settings.json'
                model_path = f'saved_models/{folder}'
                #create a folder if doesn't exist
                os.makedirs(model_path, exist_ok=True)
                with open(f'{model_path}/{file_settings_name}', 'w') as f:
                    json.dump(lstm_settings_dict, f)
                    
                optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=learning_rate, name=optimizer_name)
                #optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

                model = build_model_lstm_3(input_shape, cells_list, ifDense=ifDense, ifDropout=False, many_to_many=many_to_many)
                model.stateful = stateful
                model.compile(loss=loss, 
                              optimizer=optimizer, )
                              #metrics=metrics)
                if ifSummary:
                    model.summary() 
                    
                
                #define callbacks
                # Write TensorBoard logs
                log_dir = f'./tensorboard/{folder}/logs/{model_name}'
                tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
                #Stop training when no improvement
                early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                                  patience=patience_early_stop, 
                                                                  restore_best_weights=True)
                #Reduce learning rate when a metric has stopped improving
                reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=patience_lr)
                callbacks = [tensorboard, early_stopping, reduce_lr]
                
                model.fit(
                    x_train_lstm,
                    y_train_lstm,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=validation_split,
                    callbacks=callbacks,
                )
                
                model.save_weights(f'{model_path}/{model_name}/{model_name}')               

model_lstm_win_6_1_ovrp_0_un_5_c_10
Epoch 1/2000
5129/5129 [==============================] - 10s 2ms/step - loss: 0.0281 - val_loss: 0.0157 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0096 - val_loss: 0.0056 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0045 - val_loss: 0.0040 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0033 - val_loss: 0.0033 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0029 - val_loss: 0.0030 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0027 - val_loss: 0.0028 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0025 - val_loss: 0.0026 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0023 - val_loss: 0.0025 - lr: 0.1000
Epoch 9/2000
5129/5129 [===

4990/4990 [==============================] - 12s 2ms/step - loss: 0.0024 - val_loss: 0.0026 - lr: 1.0000e-03
Epoch 30/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0024 - val_loss: 0.0026 - lr: 1.0000e-03
Epoch 31/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0024 - val_loss: 0.0026 - lr: 1.0000e-04
Epoch 32/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0024 - val_loss: 0.0026 - lr: 1.0000e-04
Epoch 33/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0024 - val_loss: 0.0026 - lr: 1.0000e-05
Epoch 34/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0024 - val_loss: 0.0026 - lr: 1.0000e-05
Epoch 35/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0024 - val_loss: 0.0026 - lr: 1.0000e-06
Epoch 36/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0024 - val_loss: 0.0026 - lr: 1.0000e-06
Epoch 37/2000
4990/4990 [=====

Epoch 14/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0070 - val_loss: 0.0066 - lr: 0.1000
Epoch 15/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0066 - val_loss: 0.0062 - lr: 0.1000
Epoch 16/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0062 - val_loss: 0.0059 - lr: 0.1000
Epoch 17/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0059 - val_loss: 0.0056 - lr: 0.1000
Epoch 18/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0055 - val_loss: 0.0053 - lr: 0.1000
Epoch 19/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0053 - val_loss: 0.0050 - lr: 0.1000
Epoch 20/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0050 - val_loss: 0.0048 - lr: 0.1000
Epoch 21/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0048 - val_loss: 0.0046 - lr: 0.1000
Epoch 22/2000
4990/4990 [=======================

5129/5129 [==============================] - 8s 2ms/step - loss: 0.0015 - val_loss: 0.0014 - lr: 0.1000
Epoch 26/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0015 - val_loss: 0.0013 - lr: 0.1000
Epoch 27/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0015 - val_loss: 0.0013 - lr: 0.1000
Epoch 28/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0014 - val_loss: 0.0013 - lr: 0.0100
Epoch 29/2000
5129/5129 [==============================] - 9s 2ms/step - loss: 0.0014 - val_loss: 0.0013 - lr: 0.0100
Epoch 30/2000
5129/5129 [==============================] - 9s 2ms/step - loss: 0.0014 - val_loss: 0.0013 - lr: 0.0100
Epoch 31/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0014 - val_loss: 0.0013 - lr: 1.0000e-03
Epoch 32/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0014 - val_loss: 0.0013 - lr: 1.0000e-03
Epoch 33/2000
5129/5129 [==============================] - 8s 

4990/4990 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0032 - lr: 0.0100
Epoch 49/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0032 - lr: 1.0000e-03
Epoch 50/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0032 - lr: 1.0000e-03
Epoch 51/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0032 - lr: 1.0000e-04
Epoch 52/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0032 - lr: 1.0000e-04
Epoch 53/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0032 - lr: 1.0000e-05
Epoch 54/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0032 - lr: 1.0000e-05
Epoch 55/2000
4990/4990 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0032 - lr: 1.0000e-06
Epoch 56/2000
4990/4990 [=========

...

2426/2426 [==============================] - 6s 3ms/step - loss: 0.0051 - val_loss: 0.0051 - lr: 1.0000e-06
Epoch 54/2000
2426/2426 [==============================] - 6s 3ms/step - loss: 0.0051 - val_loss: 0.0051 - lr: 1.0000e-06
Epoch 55/2000
2426/2426 [==============================] - 6s 3ms/step - loss: 0.0051 - val_loss: 0.0051 - lr: 1.0000e-07
Epoch 56/2000
2426/2426 [==============================] - 6s 3ms/step - loss: 0.0051 - val_loss: 0.0051 - lr: 1.0000e-07
model_lstm_win_6_1_ovrp_0_un_10_c_20
Epoch 1/2000
2357/2357 [==============================] - 5s 2ms/step - loss: 0.0426 - val_loss: 0.0298 - lr: 0.1000
Epoch 2/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0237 - val_loss: 0.0171 - lr: 0.1000
Epoch 3/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0154 - val_loss: 0.0125 - lr: 0.1000
Epoch 4/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0124 - val_loss: 0.0108 - lr: 0.1000
Epoch 5/2000
2357/235

2218/2218 [==============================] - 5s 2ms/step - loss: 0.0190 - val_loss: 0.0180 - lr: 0.1000
Epoch 14/2000
2218/2218 [==============================] - 6s 2ms/step - loss: 0.0184 - val_loss: 0.0174 - lr: 0.1000
Epoch 15/2000
2218/2218 [==============================] - 5s 2ms/step - loss: 0.0177 - val_loss: 0.0168 - lr: 0.1000
Epoch 16/2000
2218/2218 [==============================] - 6s 3ms/step - loss: 0.0171 - val_loss: 0.0161 - lr: 0.1000
Epoch 17/2000
2218/2218 [==============================] - 6s 3ms/step - loss: 0.0165 - val_loss: 0.0155 - lr: 0.1000
Epoch 18/2000
2218/2218 [==============================] - 6s 2ms/step - loss: 0.0158 - val_loss: 0.0149 - lr: 0.1000
Epoch 19/2000
2218/2218 [==============================] - 6s 2ms/step - loss: 0.0152 - val_loss: 0.0143 - lr: 0.1000
Epoch 20/2000
2218/2218 [==============================] - 6s 3ms/step - loss: 0.0147 - val_loss: 0.0138 - lr: 0.1000
Epoch 21/2000
2218/2218 [==============================] - 6s 3ms/step

Epoch 7/2000
2495/2495 [==============================] - 4s 2ms/step - loss: 0.0079 - val_loss: 0.0072 - lr: 0.1000
Epoch 8/2000
2495/2495 [==============================] - 4s 2ms/step - loss: 0.0074 - val_loss: 0.0068 - lr: 0.1000
Epoch 9/2000
2495/2495 [==============================] - 4s 2ms/step - loss: 0.0070 - val_loss: 0.0065 - lr: 0.1000
Epoch 10/2000
2495/2495 [==============================] - 4s 2ms/step - loss: 0.0067 - val_loss: 0.0063 - lr: 0.1000
Epoch 11/2000
2495/2495 [==============================] - 4s 2ms/step - loss: 0.0065 - val_loss: 0.0060 - lr: 0.1000
Epoch 12/2000
2495/2495 [==============================] - 4s 2ms/step - loss: 0.0062 - val_loss: 0.0057 - lr: 0.1000
Epoch 13/2000
2495/2495 [==============================] - 4s 2ms/step - loss: 0.0059 - val_loss: 0.0055 - lr: 0.1000
Epoch 14/2000
2495/2495 [==============================] - 4s 2ms/step - loss: 0.0057 - val_loss: 0.0053 - lr: 0.1000
Epoch 15/2000
2495/2495 [==============================] - 

Epoch 17/2000
2426/2426 [==============================] - 6s 3ms/step - loss: 0.0137 - val_loss: 0.0127 - lr: 0.1000
Epoch 18/2000
2426/2426 [==============================] - 6s 2ms/step - loss: 0.0131 - val_loss: 0.0122 - lr: 0.1000
Epoch 19/2000
2426/2426 [==============================] - 6s 3ms/step - loss: 0.0126 - val_loss: 0.0118 - lr: 0.1000
Epoch 20/2000
2426/2426 [==============================] - 6s 2ms/step - loss: 0.0122 - val_loss: 0.0114 - lr: 0.1000
Epoch 21/2000
2426/2426 [==============================] - 6s 3ms/step - loss: 0.0117 - val_loss: 0.0110 - lr: 0.1000
Epoch 22/2000
2426/2426 [==============================] - 6s 3ms/step - loss: 0.0113 - val_loss: 0.0106 - lr: 0.1000
Epoch 23/2000
2426/2426 [==============================] - 6s 2ms/step - loss: 0.0108 - val_loss: 0.0101 - lr: 0.1000
Epoch 24/2000
2426/2426 [==============================] - 6s 3ms/step - loss: 0.0104 - val_loss: 0.0098 - lr: 0.1000
Epoch 25/2000
2426/2426 [==============================]

Epoch 16/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0063 - val_loss: 0.0067 - lr: 0.1000
Epoch 17/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0059 - val_loss: 0.0064 - lr: 0.1000
Epoch 18/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0057 - val_loss: 0.0061 - lr: 0.1000
Epoch 19/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0054 - val_loss: 0.0059 - lr: 0.1000
Epoch 20/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0052 - val_loss: 0.0056 - lr: 0.1000
Epoch 21/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0050 - val_loss: 0.0054 - lr: 0.1000
Epoch 22/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0048 - val_loss: 0.0052 - lr: 0.1000
Epoch 23/2000
2357/2357 [==============================] - 4s 2ms/step - loss: 0.0046 - val_loss: 0.0050 - lr: 0.1000
Epoch 24/2000
2357/2357 [==============================]

In [ ]:
#read data json
file_settings_name = f'{model_name}_settings.json'
file_settings_path = f'saved_models/{folder}/{file_settings_name}'
with open(file_settings_path, 'r') as f:
    lstm_settings_dict_test = json.load(f)

In [11]:
#tensorboard results
folder = 'lstm_window'
cells_list = [10,10,10]
cells_list_str = '_'.join(str(x) for x in cells_list)


model_name = f'model_lstm_window_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'

log_dir = f'./tensorboard/{folder}/logs/{model_name}'
%tensorboard --logdir $log_dir